In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/widsdatathon2020/training_v2.csv
/kaggle/input/widsdatathon2020/samplesubmission.csv
/kaggle/input/widsdatathon2020/unlabeled.csv
/kaggle/input/widsdatathon2020/WiDS Datathon 2020 Dictionary.csv
/kaggle/input/widsdatathon2020/solution_template.csv


In [15]:
from keras.models import Sequential
from keras.layers import Dense
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [3]:
train= pd.read_csv("/kaggle/input/widsdatathon2020/training_v2.csv", header=0)
test = pd.read_csv("/kaggle/input/widsdatathon2020/unlabeled.csv")

In [4]:
train_categorical_cols = []
train_numerical_cols = []
for c in  train.columns:
    if(train[c].dtype=='object'):
        train_categorical_cols.append(c)
    else:
        train_numerical_cols.append(c)
test_categorical_cols = []
test_numerical_cols = []
for c in  test.columns:
    if(test[c].dtype=='object'):
        test_categorical_cols.append(c)
    else:
        test_numerical_cols.append(c)

print(train_categorical_cols)
print(len(train_categorical_cols))
print(len(train_numerical_cols))
print(test_categorical_cols)
print(len(test_categorical_cols))
print(len(test_numerical_cols))

['ethnicity', 'gender', 'hospital_admit_source', 'icu_admit_source', 'icu_stay_type', 'icu_type', 'apache_3j_bodysystem', 'apache_2_bodysystem']
8
178
['ethnicity', 'gender', 'hospital_admit_source', 'icu_admit_source', 'icu_stay_type', 'icu_type', 'apache_3j_bodysystem', 'apache_2_bodysystem']
8
178


In [5]:
train_numerical_cols.remove('hospital_death')
test_numerical_cols.remove('hospital_death')

In [6]:
train[train_categorical_cols] = train[train_categorical_cols].fillna("")
test[test_categorical_cols] = test[test_categorical_cols].fillna("")

In [7]:
train.fillna(train.mean(), inplace=True)

In [8]:
test.fillna(test.mean(), inplace=True)

In [11]:
std_scale = StandardScaler().fit(train[train_numerical_cols])
train[train_numerical_cols] = std_scale.transform(train[train_numerical_cols])

In [12]:
train_allcols = train.columns.values.tolist()
test_allcols = test.columns.values.tolist()
allcols = train.columns.values.tolist()
inputcols = allcols[4:]
inputcols.remove('readmission_status')
outputcols = allcols[3]
print(outputcols)

hospital_death


In [13]:
X = train[inputcols]
y = train[outputcols]

In [16]:
label = LabelEncoder()
for c in  X.columns:
    if(X[c].dtype == 'object'):
        X[c]=label.fit_transform(X[c])
    else:
        X[c]=X[c]

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [17]:
X.head()

,age,bmi,elective_surgery,ethnicity,gender,height,hospital_admit_source,icu_admit_source,icu_id,icu_stay_type,...,aids,cirrhosis,diabetes_mellitus,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,apache_3j_bodysystem,apache_2_bodysystem
0,0.347324,-7.951567e-01,-0.474441,3,2,0.994578,5,2,-1.818248,0,...,-0.029405,-0.12676,1.862175,-0.115168,-0.164559,-0.084689,-0.064666,-0.145734,10,1
1,0.896647,-2.174940e-01,-0.474441,3,1,-0.899694,5,2,-1.826982,0,...,-0.029405,-0.12676,1.862175,-0.115168,-0.164559,-0.084689,-0.064666,-0.145734,9,7
2,-2.277221,3.404616e-01,-0.474441,3,1,0.285392,4,1,-1.813881,0,...,-0.029405,-0.12676,-0.541226,-0.115168,-0.164559,-0.084689,-0.064666,-0.145734,6,4
3,1.140791,-8.062419e-01,2.107744,3,1,-0.423793,9,3,-1.818248,0,...,-0.029405,-0.12676,-0.541226,-0.115168,-0.164559,-0.084689,-0.064666,-0.145734,1,1
4,-2.643436,-8.051550e-14,-0.474441,3,2,1.713095,0,1,-1.822615,0,...,-0.029405,-0.12676,-0.541226,-0.115168,-0.164559,-0.084689,-0.064666,-0.145734,11,8


In [19]:
y.head()

0    0
1    0
2    0
3    0
4    0
Name: hospital_death, dtype: int64

In [20]:
len(X.columns)

181

In [23]:
classifier = Sequential()
#First Hidden Layer
classifier.add(Dense(4, activation='relu', kernel_initializer='random_normal', input_dim=181))
#Second  Hidden Layer
classifier.add(Dense(4, activation='relu', kernel_initializer='random_normal'))
#Output Layer
classifier.add(Dense(1, activation='sigmoid', kernel_initializer='random_normal'))

In [24]:
#Compiling the neural network
classifier.compile(optimizer ='adam',loss='binary_crossentropy', metrics =['accuracy'])

In [25]:
classifier.fit(X,y, batch_size=10, epochs=200)

Epoch 1/200
91713/91713 [==============================] - 31s 336us/step - loss: 0.2187 - accuracy: 0.9168
Epoch 2/200
91713/91713 [==============================] - 28s 309us/step - loss: 0.2044 - accuracy: 0.9266
Epoch 3/200
91713/91713 [==============================] - 30s 324us/step - loss: 0.2018 - accuracy: 0.9271
Epoch 4/200
91713/91713 [==============================] - 29s 313us/step - loss: 0.2001 - accuracy: 0.9273
Epoch 5/200
91713/91713 [==============================] - 30s 327us/step - loss: 0.1992 - accuracy: 0.9278
Epoch 6/200
91713/91713 [==============================] - 29s 316us/step - loss: 0.1984 - accuracy: 0.9274
Epoch 7/200
91713/91713 [==============================] - 30s 328us/step - loss: 0.1978 - accuracy: 0.9277
Epoch 8/200
91713/91713 [==============================] - 29s 312us/step - loss: 0.1974 - accuracy: 0.9280
Epoch 9/200
91713/91713 [==============================] - 30s 325us/step - loss: 0.1968 - accuracy: 0.9285
Epoch 10/200
91713/91713 [==

91713/91713 [==============================] - 30s 325us/step - loss: 0.1909 - accuracy: 0.9299
Epoch 77/200
91713/91713 [==============================] - 30s 330us/step - loss: 0.1908 - accuracy: 0.9297
Epoch 78/200
91713/91713 [==============================] - 30s 326us/step - loss: 0.1909 - accuracy: 0.9297
Epoch 79/200
91713/91713 [==============================] - 30s 329us/step - loss: 0.1910 - accuracy: 0.9300
Epoch 80/200
91713/91713 [==============================] - 30s 324us/step - loss: 0.1909 - accuracy: 0.9297
Epoch 81/200
91713/91713 [==============================] - 31s 335us/step - loss: 0.1908 - accuracy: 0.9294
Epoch 82/200
91713/91713 [==============================] - 30s 328us/step - loss: 0.1907 - accuracy: 0.9296
Epoch 83/200
91713/91713 [==============================] - 31s 337us/step - loss: 0.1909 - accuracy: 0.9299
Epoch 84/200
91713/91713 [==============================] - 30s 324us/step - loss: 0.1907 - accuracy: 0.9302
Epoch 85/200
91713/91713 [======

91713/91713 [==============================] - 30s 323us/step - loss: 0.1901 - accuracy: 0.9302
Epoch 151/200
91713/91713 [==============================] - 30s 322us/step - loss: 0.1901 - accuracy: 0.9303
Epoch 152/200
91713/91713 [==============================] - 30s 325us/step - loss: 0.1902 - accuracy: 0.9297
Epoch 153/200
91713/91713 [==============================] - 30s 322us/step - loss: 0.1901 - accuracy: 0.9298
Epoch 154/200
91713/91713 [==============================] - 30s 331us/step - loss: 0.1901 - accuracy: 0.9301
Epoch 155/200
91713/91713 [==============================] - 30s 325us/step - loss: 0.1901 - accuracy: 0.9298
Epoch 156/200
91713/91713 [==============================] - 30s 331us/step - loss: 0.1900 - accuracy: 0.9301
Epoch 157/200
91713/91713 [==============================] - 29s 320us/step - loss: 0.1901 - accuracy: 0.9300
Epoch 158/200
91713/91713 [==============================] - 30s 325us/step - loss: 0.1901 - accuracy: 0.9300
Epoch 159/200
91713/9171

In [28]:
X_test = test[inputcols]

In [29]:
len(X_test.columns)

181

In [30]:
label = LabelEncoder()
for c in  X_test.columns:
    if(X_test[c].dtype == 'object'):
        X_test[c]=label.fit_transform(X_test[c])
    else:
        X_test[c]=X_test[c]

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [32]:
y_pred=classifier.predict(X_test)

In [34]:
len(y_pred)

39308

In [35]:
test["hospital_death"].unique()

array([nan])

In [36]:
test["hospital_death"] = y_pred
test[["encounter_id","hospital_death"]].to_csv("submission_keras1.csv",index=False)

In [ ]:
print('completed')